In [2]:
import requests
import pandas as pd
import collect_data 
import pickle

# Questions

1. Which price is better?  Open price, close price, high price or low price?
2. When should I use log in linear model?

# Load data 

### NFT market

[x] primary sale (PrimarySale),  
[ ] secondary sale,  
[x] primary sale USD,  
[ ] secondary sale USD,  
[ ] and control for the changes in the number of unique wallets (ΔNumWallets),   
[ ] the number of buyers (ΔNumBuyers),   
[ ] the number of sellers (ΔNumSellers),   
[ ] the number of sales (ΔNumSales),   
[ ] the sales volume in USD (ΔSalesUSD),   
[ ] ETHUSD exchange rate (ΔETHUSD), 
[ ] the ETH trading volume (ΔETHVol)    
[ ] Google Index: worldwide attention to Ethereum (Adj. SVI) 

In [7]:
df_NFT = pd.read_csv('data/Data_NFT.csv')
df_NFT["DateTime"] = pd.to_datetime(df_NFT["DateTime"])

In [17]:
def color_corr_level(val):
    val = abs(val)
    if val>0.8:
        color = 'red'
    elif val>0.3:
        color = 'orange'
    else:
        color = 'black'
    return 'color:%s'%color

In [18]:
df_NFT.corr().style.applymap(color_corr_level)

,Average USD,Active market wallets,Number of sales,Sales USD,Primary Sales,Secondary sales,Primary sales USD,Secondary sales USD,Unique buyers,Unique sellers
Average USD,1,-0.056401,-0.196325,0.589376,-0.232878,-0.150553,0.426496,0.612942,-0.0759741,-0.0686643
Active market wallets,-0.056401,1,0.931859,0.624827,0.789492,0.976901,0.46159,0.645029,0.994838,0.990561
Number of sales,-0.196325,0.931859,1,0.543866,0.948137,0.964535,0.455396,0.534284,0.952755,0.905275
Sales USD,0.589376,0.624827,0.543866,1,0.497343,0.539788,0.866653,0.967522,0.632083,0.579748
Primary Sales,-0.232878,0.789492,0.948137,0.497343,1,0.83061,0.476991,0.457901,0.834144,0.737341
Secondary sales,-0.150553,0.976901,0.964535,0.539788,0.83061,1,0.401701,0.555754,0.976428,0.973635
Primary sales USD,0.426496,0.46159,0.455396,0.866653,0.476991,0.401701,1,0.712389,0.491352,0.394306
Secondary sales USD,0.612942,0.645029,0.534284,0.967522,0.457901,0.555754,0.712389,1,0.640156,0.61571
Unique buyers,-0.0759741,0.994838,0.952755,0.632083,0.834144,0.976428,0.491352,0.640156,1,0.974469
Unique sellers,-0.0686643,0.990561,0.905275,0.579748,0.737341,0.973635,0.394306,0.61571,0.974469,1


### External indice

In [40]:
df_ETH = pd.read_csv('data/ETH-USD.csv')
df_ETH=df_ETH.rename(columns={"Open":"Open_ETH"})
df_NASDAQ = pd.read_csv('data/NASDAQ.csv')
df_NASDAQ=df_NASDAQ.rename(columns={"Open":"Open_NASDAQ"})
df_USD = pd.read_csv('data/USD_index.csv')
df_USD=df_USD.rename(columns={"Open":"Open_USD"})
# df_ETH.rename(columns={"Open":"open_ETH", "High":"high_ETH", "Low":"low_ETH", "Close":"close_ETH", "Adj Close":"adj_close_ETH", "Volume":"volume_ETH"})

In [42]:
df_USD

,Date,Open_USD,High,Low,Close,Adj Close,Volume
0,2021-10-05,93.830002,94.070000,93.809998,93.980003,93.980003,0.0
1,2021-10-06,93.989998,94.449997,93.980003,94.269997,94.269997,0.0
2,2021-10-07,94.230003,94.279999,94.080002,94.220001,94.220001,0.0
3,2021-10-08,94.209999,94.339996,93.940002,94.070000,94.070000,0.0
4,2021-10-10,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
304,2022-09-29,113.010002,113.790001,111.919998,112.250000,112.250000,0.0
305,2022-09-30,111.750000,112.669998,111.580002,112.120003,112.120003,0.0
306,2022-10-02,NaN,NaN,NaN,NaN,NaN,NaN
307,2022-10-03,112.169998,112.540001,111.470001,111.750000,111.750000,0.0


In [27]:
df_extern = pd.concat([df_ETH[['Date',]], df_BYAC.drop('timestamp', axis=1).astype('float64')], axis=1, join='inner')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-10-05,3381.784424,3541.451660,3365.817627,3518.518555,3518.518555,16632591670
1,2021-10-06,3516.570313,3622.551270,3354.469727,3580.562012,3580.562012,21855226591
2,2021-10-07,3576.814453,3650.013184,3479.909180,3587.974854,3587.974854,19090322927
3,2021-10-08,3587.832764,3667.957520,3547.828125,3563.759277,3563.759277,16222029488
4,2021-10-09,3559.997803,3628.237305,3544.640381,3575.716797,3575.716797,12707036942
...,...,...,...,...,...,...,...
360,2022-09-30,1335.646484,1368.743408,1320.383179,1327.978638,1327.978638,14250100093
361,2022-10-01,1328.193726,1332.516479,1306.102539,1311.644409,1311.644409,6227961237
362,2022-10-02,1311.753418,1316.330078,1275.336060,1276.093506,1276.093506,7578351650
363,2022-10-03,1276.163452,1326.554443,1271.150879,1323.439209,1323.439209,10153070907


# BYAC and Cryptopunk

In [2]:
ranks = collect_data.top100_value()

In [3]:
print(ranks[3].name)
print(ranks[3].address)
BYAC_address = ranks[3].address

BoredApeYachtClub
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d


In [21]:
BYAC_data = collect_data.collection_price(contract_address=BYAC_address, endtime="2022-10-05T00:00:00Z")

In [22]:
BYAC_data_05 = collect_data.collection_price(contract_address=BYAC_address, endtime="2022-10-06T00:00:00Z")

In [23]:
BYAC_data['dataPoints'][-1]

{'timestamp': '2022-10-04T00:00:00Z',
 'min': '61.8200392304197776',
 'max': '150',
 'avg': '93.6440078460839555'}

In [24]:
BYAC_data_05['dataPoints'][-1]

{'timestamp': '2022-10-05T00:00:00Z',
 'min': '72.9775957080854495',
 'max': '100.1423550633253784',
 'avg': '84.3566502571369426'}

In [29]:
df_BYAC = pd.DataFrame(BYAC_data['dataPoints']+[BYAC_data['dataPoints'][-1]])

In [30]:
df_BYAC

,timestamp,min,max,avg
0,2021-10-05T00:00:00Z,35.269,65,46.1510344827586207
1,2021-10-06T00:00:00Z,10.5,90,41.5033411
2,2021-10-07T00:00:00Z,21.643836,99,44.6864021176470588
3,2021-10-08T00:00:00Z,31.505,53,39.02653125
4,2021-10-09T00:00:00Z,31.005,110,48.5333470588235294
...,...,...,...,...
361,2022-10-01T00:00:00Z,50.4028497096936034,777,245.2569624274234009
362,2022-10-02T00:00:00Z,72.99,144,90.392353611981722
363,2022-10-03T00:00:00Z,22.7237874628446507,112,69.2275240368939183
364,2022-10-04T00:00:00Z,61.8200392304197776,150,93.6440078460839555


In [33]:
df_BYAC.rename(columns={"min": "min_price_BYAC", "max": "max_price_BYAC", "avg":"avg_price_BYAC"})

,timestamp,min_price_BYAC,max_price_BYAC,avg_price_BYAC
0,2021-10-05T00:00:00Z,35.269,65,46.1510344827586207
1,2021-10-06T00:00:00Z,10.5,90,41.5033411
2,2021-10-07T00:00:00Z,21.643836,99,44.6864021176470588
3,2021-10-08T00:00:00Z,31.505,53,39.02653125
4,2021-10-09T00:00:00Z,31.005,110,48.5333470588235294
...,...,...,...,...
361,2022-10-01T00:00:00Z,50.4028497096936034,777,245.2569624274234009
362,2022-10-02T00:00:00Z,72.99,144,90.392353611981722
363,2022-10-03T00:00:00Z,22.7237874628446507,112,69.2275240368939183
364,2022-10-04T00:00:00Z,61.8200392304197776,150,93.6440078460839555


In [73]:
df_BYAC.drop('timestamp', axis=1).astype('float64')

,min,max,avg
0,35.269000,65.0,46.151034
1,10.500000,90.0,41.503341
2,21.643836,99.0,44.686402
3,31.505000,53.0,39.026531
4,31.005000,110.0,48.533347
...,...,...,...
361,50.402850,777.0,245.256962
362,72.990000,144.0,90.392354
363,22.723787,112.0,69.227524
364,61.820039,150.0,93.644008


In [74]:
df_all = pd.concat([df_ETH.drop(labels="Date",axis=1), df_BYAC.drop('timestamp', axis=1).astype('float64')], axis=1, join='inner')

In [76]:
df_all.corr()

,Open,High,Low,Close,Adj Close,Volume,min,max,avg
Open,1.000000,0.998307,0.996452,0.994385,0.994385,0.091593,-0.381571,-0.181816,-0.469171
High,0.998307,1.000000,0.996610,0.996945,0.996945,0.104323,-0.390260,-0.179390,-0.473976
Low,0.996452,0.996610,1.000000,0.997561,0.997561,0.048475,-0.381714,-0.178204,-0.462961
Close,0.994385,0.996945,0.997561,1.000000,1.000000,0.071375,-0.390844,-0.177929,-0.471658
Adj Close,0.994385,0.996945,0.997561,1.000000,1.000000,0.071375,-0.390844,-0.177929,-0.471658
Volume,0.091593,0.104323,0.048475,0.071375,0.071375,1.000000,-0.095454,-0.004437,-0.187451
min,-0.381571,-0.390260,-0.381714,-0.390844,-0.390844,-0.095454,1.000000,0.031778,0.474809
max,-0.181816,-0.179390,-0.178204,-0.177929,-0.177929,-0.004437,0.031778,1.000000,0.517728
avg,-0.469171,-0.473976,-0.462961,-0.471658,-0.471658,-0.187451,0.474809,0.517728,1.000000
